In [17]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import pymorphy2
from pymorphy2 import MorphAnalyzer
import sklearn.metrics

morph = MorphAnalyzer()

text_p - файл с 30 положительными отзывами; text_n - файл с 30 отрицательными отзывами; test - 10 отзывов для тестирования. Данные я закопипастила

In [8]:
with open('reviews+.txt', 'r', encoding='utf-8') as f1:
    text_p = f1.read().split('\n')

with open('reviews-.txt', 'r', encoding='utf-8') as f2:
    text_n = f2.read().split('\n')
    
with open('test.txt', 'r', encoding='utf-8') as f3:
  test = f3.read().split('\n')

Функция get_lemmas обрабатывает отзыв (токенизация, нижний регистр, лемматизация) на выходе получает список лемм рассматриваемого отзыва

In [9]:
def get_lemmas(data):
  lemmas = []
  for row in data:
    t = word_tokenize(row)
    words = [w.lower() for w in t if w.isalpha()]
    for w in words:
      w = morph.parse(w)
      lemmas.append(w[0].normal_form)
  return(lemmas)

In [10]:
pos = Counter(get_lemmas(text_p))

neg = Counter(get_lemmas(text_n))

pos_freq и neg_freq - списки лемм, которые втсречаются в отзывах чаще 2 раз

In [11]:
pos_freq = []
for i in pos.keys():
  if pos[i] > 2:
    pos_freq.append(i)
    
neg_freq = []
for i in neg.keys():
  if neg[i] > 2:
    neg_freq.append(i)

Сравниваю 2 списка и оставляю в каждом только уникальные для его тональности леммы

In [12]:
for i in pos_freq:
  if i in neg_freq:
    pos_freq.remove(i)
    neg_freq.remove(i)

Функция get_result сначала обрабатывает текст отзыва, а потом определяет его тональность

In [13]:
def get_result(data):
  lemmas = []
  d = word_tokenize(data)
  words = [w.lower() for w in d if w.isalpha()]
  for w in words:
    w = morph.parse(w)
    lemmas.append(w[0].normal_form)
  lemmas = Counter(lemmas)
  p = 0
  n = 0
  for i in lemmas.keys():
    if i in pos_freq:
      p += lemmas[i]
    elif i in neg_freq:
      n += lemmas[i]
  if n > p:
    return("negative")
  elif n < p:
    return("positive")
  else:
    return("neutral")

Считаю accuracy

In [14]:
y_true = ['positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative']
y_pred = []

for row in test:
  y_pred.append(get_result(row))

In [16]:
acc = sklearn.metrics.accuracy_score(y_true, y_pred)
acc

0.5

Accuracy получилась маленькой, что можно сделать:
- на этапе начальной обработки текста отзыва убрать стоп-слова
- учитываь не отдельные леммы, а биграммы и триграммы, так как, например, для отрицательных отзывов важно учитывать отрицательную форму глагола (НЕ + глагол), а не частицу и глагол по отдельности
- присвоить некоторым словам бОльший вес при подсчете переменных p и n (например, слово "обожаю" явно для нас ценнее, чем слово "фильм")